# Redes neurais profundas

Nos notebooks anteriores (`introducao.ipynb` e `not_mnist_with_tf.ipynb`) você aprendeu como construir uma rede neural com uma camada. Neste notebook você vai aprender a construir uma rede neural de várias camadas com o TensorFlow. Adicionar uma camada oculta a uma rede permite que ela modele funções mais complexas. Além disso, usar uma função de ativação não linear na camada oculta permite que ela modele funções não lineares.

Este notebook foi construído a partir dos conhecimentos saprensentados na aula `Introdução às redes neurais`. Conceitos como feedforward e dropout são alguns dos muitos cruciais para o entendimento completo deste notebook.

---
## ReLu

A primeira coisa que nós vamos aprender a implementar no TensorFlow é a camada oculta com função de ativação ReLu (Rectified Linear Unit, ou unidade retificada linear). A ReLu é uma função não linear que retorna 0 para entradas negativas e $x$ para todas as entradas $x > 0$.

O TensorFlow fornece a função ReLu como [**`tf.nn.relu()`**](https://www.tensorflow.org/api_docs/python/tf/nn/relu), como visto no trecho de código abaixo:

```
# Camada oculta com função de ativação ReLu
hidden_layer = tf.add(tf.matmul(X, hidden_w), hidden_b)
hidden_layer = tf.nn.relu(hidden_layer)

output = tf.add(tf.matmul(hidden_layer, output_w), output_b)
```

O trecho acima aplica a função [**`tf.nn.relu()`**](https://www.tensorflow.org/api_docs/python/tf/nn/relu) na camada `hidden_layer`, o que ba prática desativa quaisquer pesos negativos e funciona como chave liga/desliga. Camadas adicionais, como a camada `output`, aplicadas após a ativação transformam o modelo em uma função não-linear. Essa não-linearidade permite que a rede resolva problemas mais complexos.

### Implementação

<img src='img/two_layer_network.png' width=600px>


Na célula abaixo, você vai usar a função ReLu para transformar uma rede linear de uma camada em uma rede não-linear multicamadas.

In [1]:
import tensorflow as tf

output = None
hidden_layer_weights = [[0.1, 0.2, 0.4],
                        [0.4, 0.6, 0.6],
                        [0.5, 0.9, 0.1],
                        [0.8, 0.2, 0.8]]
out_weights = [[0.1, 0.6],
               [0.2, 0.1],
               [0.7, 0.9]]

# Weights and biases
weights = [tf.Variable(hidden_layer_weights),
           tf.Variable(out_weights)]
biases = [tf.Variable(tf.zeros(3)),
          tf.Variable(tf.zeros(2))]

# Input
features = tf.Variable([[1.0, 2.0, 3.0, 4.0], 
                        [-1.0, -2.0, -3.0, -4.0], 
                        [11.0, 12.0, 13.0, 14.0]])

# TODO: Create Model
hidden_layer = tf.add(tf.matmul(features, weights[0]), biases[0])
hidden_layer = tf.nn.relu(hidden_layer)
output = tf.add(tf.matmul(hidden_layer, weights[1]), biases[1])

# TODO: Print session results
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(output))

[[ 5.11      8.440001]
 [ 0.        0.      ]
 [24.010002 38.239998]]


---
## Redes neurais profundas no TensorFlow

Você viu anteriormente como criar um classificador logístico usando TensorFlow. Agora você vai ver como usar o classificador logístico para construir uma rede neural profunda.

No passo a passo a seguir, vamos analisar um código escrito usando TensorFlow para classificar letras no conjunto de dados MNIST.

### Obtendo o conjunto de dados

Você vai usar o conjunto de dados MNIST fornecido pelo TensorFlow, que agrupa em lotes e codifica os dados usando a codificação one-hot.

In [4]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('.', one_hot=True, reshape=False)

Extracting ./train-images-idx3-ubyte.gz
Extracting ./train-labels-idx1-ubyte.gz
Extracting ./t10k-images-idx3-ubyte.gz
Extracting ./t10k-labels-idx1-ubyte.gz


### Definindo hiperparâmetros

O foco aqui é dado à arquitetura da rede neural multicamadas, não ao ajuste de hiperparâmetros, então dessa vez vamos simplesmente fornecer os hiperparâmetros. 

In [5]:
import tensorflow as tf

learning_rate = 0.001
epochs = 20
batch_size = 128
display_step = 1

n_input = 28 * 28    # Dados de entrada do MNIST (imagens 28x28)
n_classes = 10       # Total de classes do MNIST (algarismos de 0 a 9)

### Hiperparâmetros da camada oculta

A variável `n_hidden_layer` determina o tamanho da camada oculta na rede neural. Esse valor também é conhecido como a largura de uma camada.

In [6]:
n_hidden_layer = 256

### Pesos e vieses

Redes neurais profundas usam múltiplas camadas, sendo que cada camada exige seus próprios pesos e vieses. No código abaixo, os pesos e vieses da camada `hidden_layer` pertencem à camada oculta, e os pesos e vieses da camada `output` pertencem à camda de saída.

In [7]:
weights = {'hidden_layer': tf.Variable(tf.random_normal([n_input, n_hidden_layer])),
           'output': tf.Variable(tf.random_normal([n_hidden_layer, n_classes]))}
biases = {'hidden_layer': tf.Variable(tf.random_normal([n_hidden_layer])),
          'output': tf.Variable(tf.random_normal([n_classes]))}

Se a rede neural fosse mais profunda, haveria pesos e vieses para cada camada adicional.

### Input

Os dados do conjunto MNIST são formados por imagens 28px por 28px em um único [canal](https://en.wikipedia.org/wiki/Channel_(digital_image%29). Na célula abaixo, a função [**`tf.reshape`**](https://www.tensorflow.org/api_docs/python/tf/reshape) remodela as matrizes 28px por 28px presentes em `x`, transformando-as em vetores 784px por 1px.

In [8]:
x = tf.placeholder(tf.float32, [None, 28, 28, 1])
y = tf.placeholder(tf.float32, [None, n_classes])

x_flat = tf.reshape(x, [-1, n_input])

### Percepton multicamadas

<img src='img/mlp.png' width=500px>

Você viu a função linear `Xw + b` antes. A combinação linear de funções usando uma ReLu gera uma rede de duas camadas.

In [9]:
# Hidden layer with ReLu activation
layer_1 = tf.add(tf.matmul(x_flat, weights['hidden_layer']), biases['hidden_layer'])
layer_1 = tf.nn.relu(layer_1)

# Output layer with linear activation
logits = tf.add(tf.matmul(layer_1, weights['output']), biases['output'])

### Otimização

A técnica utilizada será a mesma utilizada no notebook `introducao.ipynb`.

In [10]:
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(loss)

### Sessão

A biblioteca MNIST no TensorFlow fornece a possibilidade de receber o conjunto de dados em lotes. A função `mnist.train.next_batch()` retorna um subconjunto dos dados de treinamento, como visto a seguir:

In [11]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(epochs):
        total_batch = int(mnist.train.num_examples / batch_size)
        for _ in range(total_batch):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            sess.run(optimizer, feed_dict={x: batch_x, y: batch_y})

---
## Redes neurais mais profundas

<img src='img/layers.png' width=500px>

Adicionar mais camadas à rede permite que você resolva problemas mais complicados.